In [2]:
%load_ext autoreload

%autoreload 2

import gauleg as gl 
import numpy as np 

In [21]:
# initial variables 

a = 0 
b = 1 
l = 2 # for a 4 slot problem 
h = 2 **(-l) # distance between xi, xi+1 is 1 * 2**-l 
n1 = 2**l + 1 # number of nodes 

n2 = 5 # number of roots in each GL 
ijlist = list(range(2**l-1))

dim = 2**l -1 # number of phi functions i have for l 

### sanity check ###
"""
if l = 2 i expect to have 4 slots and 3 functions and 5 points 
2**l = 2**2 = 4 --> [a = x0,x1,x2,x3,x4 = b]
dim of matrix = 3 = 2**2 -1 
gradient = 1 / 2**-l 
n1 = 5  --> number of points 
"""


x_list = gl.listi(a,b,h,n1)
print(x_list)


a = gl.a(0.25)
print(a)

matrix = np.zeros((dim, dim))
print(matrix)






[0.0, 0.25, 0.5, 0.75, 1.0]
2.2474039592545227
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [4]:
results = gl.gauleg(n2 = 5)
print(results)
print(results[0])


([-0.906179845938664, -0.5384693101056831, 0.0, 0.5384693101056831, 0.906179845938664], [0.236926885056189, 0.47862867049936647, 0.5688888888888889, 0.47862867049936647, 0.236926885056189])
[-0.906179845938664, -0.5384693101056831, 0.0, 0.5384693101056831, 0.906179845938664]


In [5]:
gl.GL(xi = results[0], ci = results[1], y = 0.25, x= 0, n2 = 5 )


0.5310875782893552

In [6]:
gl.phiij(numofnodes=5, i=0, j =1 , l = l , xlist=x_list)

-9.461277757124352

In [35]:
gl.matrix(ijlist=ijlist, matrix=matrix, n1=n1, l=l, xlist=x_list )

array([[ 17.95867901,  -9.46127776,   0.        ],
       [ -9.46127776,  19.79557685, -10.33429909],
       [  0.        , -10.33429909,  21.3964841 ]])

In [34]:
matrix[1][2]

np.float64(-10.334299088264828)

In [32]:
a = 0 
b = 1 
l = 2 # for a 32 slot problem 
h = 2 **(-l) # distance between xi, xi+1 is 1 * 2**-l 
n1 = 2**l + 1 # number of nodes 

n2 = 5 # number of roots in each GL 
ijlist = list(range(2**l-1))
dim = 2**l -1 # number of phi functions i have for l 
x_list = gl.listi(a,b,h,n1)
matrix = np.zeros((dim, dim))


In [33]:
gl.matrix(ijlist=ijlist, matrix=matrix, n1=n1, l=l, xlist=x_list )

array([[ 17.95867901,  -9.46127776,   0.        ],
       [ -9.46127776,  19.79557685, -10.33429909],
       [  0.        , -10.33429909,  21.3964841 ]])

In [11]:
%timeit gl.matrix(ijlist=ijlist, matrix=matrix, n1=n1, l=l, xlist=x_list )

1.08 ms ± 26.7 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
F = np.zeros((dim,1))
F[1][0]

np.float64(0.0)

In [19]:
gl.f(2)

4.570154344009936

In [40]:
f = gl.intergrand()


[[36.561234752079486]]

In [41]:

def GL(xi, ci, y, x, n2, func):
	sum = 0 
	for i in range(n2):
		prod = ci[i]*func(0.5*(((y-x)*xi[i])+(y+x)))
		sum = sum + prod 
	return (y-x)/2 * (sum)

Fj = GL( xi = gl.gauleg(n2)[0], ci = gl.gauleg(n2)[1], y = 1, x = 0, n2 = n2, func = f() )
Fj

TypeError: intergrand.<locals>.phi_j() missing 3 required positional arguments: 'numofnodes', 'i', and 'l'